In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

In [2]:
def logloss(act, preds):
    epsilon = 1e-15
    preds = sp.maximum(epsilon, preds)
    preds = sp.minimum(1 - epsilon, preds)
    ll = sum(act * sp.log(preds) + sp.subtract(1, act) * sp.log(sp.subtract(1, preds)))
    ll = ll * -1.0 / len(act)
    return ll

In [17]:
drop_cols = ['hourClick_min',
             'hourClick_median',
             'category_0_sum_count',
             'is_train',
             'item_price_level_16',
             'item_price_level_17',
             'item_pv_level_1',
             'shop_review_num_level_0',
             'shop_review_num_level_1',
             'shop_star_level_4999',
             'category_0_7908382889764677758']
train = pd.read_pickle("../数据/model/_5_merge_data_train.pickle")
train = train.set_index("instance_id")
val = pd.read_pickle("../数据/model/_5_merge_data_val.pickle")
val = val.set_index("instance_id")
test = pd.read_pickle("../数据/model/_5_merge_data_test.pickle")
test = test.set_index("instance_id")
train = train.drop(drop_cols,axis=1)
test = test.drop(drop_cols,axis=1)
val = val.drop(drop_cols,axis=1)

In [19]:
X_tr = train.drop(["is_trade"],axis=1)
y_tr = train["is_trade"]
X_val = val.drop(["is_trade"],axis=1)
y_val = val["is_trade"]

In [26]:
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=29, max_depth=3, learning_rate=0.01, n_estimators=10000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=1, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, nthread=-1, silent=True)

In [27]:
model.fit(X_tr, y_tr, eval_metric='logloss',eval_set=[(X_tr, y_tr), (X_val, y_val)],early_stopping_rounds=100)

/home/zhh/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:278: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in 2.0.12 version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/home/zhh/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:281: LGBMDeprecationWarning: The `nthread` parameter is deprecated and will be removed in 2.0.12 version. Please use `n_jobs` instead.
  'Please use `n_jobs` instead.', LGBMDeprecationWarning)
/home/zhh/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/home/zhh/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be remov

[1]	training's binary_logloss: 0.6839	valid_1's binary_logloss: 0.683874
Training until validation scores don't improve for 100 rounds.
[2]	training's binary_logloss: 0.674835	valid_1's binary_logloss: 0.674795
[3]	training's binary_logloss: 0.665948	valid_1's binary_logloss: 0.665894
[4]	training's binary_logloss: 0.657235	valid_1's binary_logloss: 0.657156
[5]	training's binary_logloss: 0.648689	valid_1's binary_logloss: 0.648597
[6]	training's binary_logloss: 0.640306	valid_1's binary_logloss: 0.640202
[7]	training's binary_logloss: 0.632083	valid_1's binary_logloss: 0.631954
[8]	training's binary_logloss: 0.624013	valid_1's binary_logloss: 0.623873
[9]	training's binary_logloss: 0.616094	valid_1's binary_logloss: 0.615942
[10]	training's binary_logloss: 0.608321	valid_1's binary_logloss: 0.608155
[11]	training's binary_logloss: 0.600691	valid_1's binary_logloss: 0.600501
[12]	training's binary_logloss: 0.5932	valid_1's binary_logloss: 0.592998
[13]	training's binary_logloss: 0.5858

/home/zhh/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:278: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in 2.0.12 version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/home/zhh/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:281: LGBMDeprecationWarning: The `nthread` parameter is deprecated and will be removed in 2.0.12 version. Please use `n_jobs` instead.
  'Please use `n_jobs` instead.', LGBMDeprecationWarning)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.01,
        max_bin=425, max_depth=3, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=10000, n_jobs=-1, nthread=-1,
        num_leaves=29, objective='binary', random_state=None, reg_alpha=3,
        reg_lambda=5, seed=1000, silent=True, subsample=1,
        subsample_for_bin=50000, subsample_freq=1)

In [28]:
y_pred = model.predict(X_val)

/home/zhh/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
pd.Series(y_pred).value_counts()

0    57406
dtype: int64

In [53]:
proba = model.predict_proba(test)

In [54]:
result_v1 = pd.DataFrame(test.index)
result_v1["predicted_score"] = proba[:,-1]

In [61]:
result_v1.to_csv("../数据/model/_6_result_v1_04082335.txt",encoding="utf-8",sep=" ",index=False)

In [58]:
test = pd.read_csv("../数据/raw_path/round1_ijcai_18_test_a_20180301.txt",sep=" ")

In [60]:
test.shape

(18371, 26)